In [46]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense , merge
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import ReduceLROnPlateau
from keras.layers.merge import dot
from keras.models import Model
from keras.layers import Dropout, Flatten,Activation,Input,Embedding
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
keras.__version__

'2.2.0'

In [10]:
path = os.path.join(os.path.abspath(''), "MovieLens/ml-25m")
movies = pd.read_csv(os.path.join(path, 'movies.csv'))
rating=pd.read_csv(os.path.join(path, "ratings.csv"))

In [11]:
rating.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [12]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [13]:
user_id = np.unique(rating['userId'])
movie_id = np.unique(rating['movieId'])

In [14]:
high_movies = movies.copy()
high_movies['genres'] = high_movies.genres.str.split('|')
high_movies.head()


,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [15]:
movie_genre = high_movies.copy()

for index, row in movies.iterrows():
    for genre in row['genres']:
        movie_genre.at[index, genre] = 1
movie_genre = movie_genre.fillna(0)
movie_genre.head()

,movieId,title,genres,A,d,v,e,n,t,u,...,S,-,I,X,W,N,(,,g,)
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),[Comedy],0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
movie_genre=movie_genre.drop(columns=['genres'])
movie_genre.head()

,movieId,title,A,d,v,e,n,t,u,r,...,S,-,I,X,W,N,(,,g,)
0,1,Toy Story (1995),1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [26]:
index=sorted(list(user_id[0:100]))
columns=sorted(list(movie_id[0:100]))
 
joined = pd.pivot_table(data=rating[0:100],values='rating',index='userId',columns='movieId')
joined

movieId,1,62,110,150,151,236,260,261,266,296,...,8685,8729,8786,8873,8973,27193,27266,27721,31956,32591
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,1.0,3.5,4.0,3.0,4.0,3.0,4.5,3.0,3.5,5.0
2,3.5,0.5,5.0,4.0,4.5,4.0,5.0,0.5,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
split = np.random.rand(len(rating)) < 0.85
train = rating[split]
valid = rating[~split]

(25000095,)

In [36]:
n_latent = 64
n_user = len(user_id)
n_movie = len(movie_id)

In [37]:
user_input=Input(shape=(1,),name='user_input',dtype='int64')

In [38]:
user_embedding=Embedding(n_user,n_latent,name='user_embedding')(user_input)

In [39]:
user_vec =Flatten(name='FlattenUsers')(user_embedding)

In [41]:
movie_input=Input(shape=(1,),name='movie_input',dtype='int64')
movie_embedding=Embedding(n_movie,n_latent,name='movie_embedding')(movie_input)
movie_vec=Flatten(name='FlattenMovies')(movie_embedding)

In [42]:
sim=dot([user_vec,movie_vec],name='Simalarity-Dot-Product',axes=1)
model =keras.models.Model([user_input, movie_input],sim)

In [43]:
model.compile(optimizer=Adam(lr=1e-4),loss='mse')


In [44]:
train.shape
batch_size=128
epochs=50

In [45]:
History = model.fit([train.userId,train.movieId],train.rating, batch_size=batch_size,
                              epochs =epochs, validation_data = ([valid.userId,valid.movieId],valid.rating),
                              verbose = 1)

Train on 21252853 samples, validate on 3747242 samples
Epoch 1/50
21252853/21252853 [==============================] - 29828s 1ms/step - loss: 599.8854 - val_loss: 898.2916
Epoch 2/50
 3259648/21252853 [===>..........................] - ETA: 6:50:47 - loss: 867.3556

KeyboardInterrupt: 

In [ ]:
plt.plot(History.history['loss'] , 'g')
plt.plot(History.history['val_loss'] , 'b')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid(True)
plt.show()